In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline
pipe = pipeline("summarization",model="facebook/bart-large-cnn")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cpu


In [ ]:
pipe("""
Electric vehicles (EVs) have seen a dramatic rise in popularity over
 the last decade. With growing concerns over climate change and rising
 fuel costs, consumers and governments alike have begun to favor cleaner,
 more sustainable transportation options. EVs, which run on electricity rather
  than gasoline or diesel, produce significantly fewer emissions and offer
   long-term savings on fuel and maintenance.

Major car manufacturers are investing heavily in EV technology,
and many countries are offering incentives to encourage people to switch.
 However, challenges remain, such as the need for better battery technology,
 more charging stations, and increased public awareness. Despite these obstacles,
  experts believe that the future of transportation is electric, with EV adoption
   expected to grow exponentially in the coming years.""")

[{'summary_text': 'Electric vehicles (EVs) have seen a dramatic rise in popularity over the last decade. With growing concerns over climate change and rising fuel costs, consumers and governments alike have begun to favor cleaner, more sustainable transportation options. However, challenges remain, such as the need for better battery technology, and more charging stations.'}]

In [ ]:
#sentiment analysis
sentiment_analyser=pipeline("text-classification",model="nlptown/bert-base-multilingual-uncased-sentiment")

sentiment_analyser("I HATE IT.")

Device set to use cpu


[{'label': '1 star', 'score': 0.7190806865692139}]

In [ ]:
#sentiment analysis
sentiment_analyser=pipeline("text-classification",model="nlptown/bert-base-multilingual-uncased-sentiment")

sentiment_analyser("I LOVE IT.")

Device set to use cpu


[{'label': '5 stars', 'score': 0.8433926105499268}]

In [ ]:
#used another model
sentiment_analyser1=pipeline("sentiment-analysis",model="distilbert-base-uncased-finetuned-sst-2-english")
sentiment_analyser1("I HATE IT.")

Device set to use cpu


[{'label': 'NEGATIVE', 'score': 0.9996691942214966}]

In [ ]:
#news agency
#you get a news, you have to segregate the news to tell whether it is around education, technology or lifestyle

classification  = pipeline("zero-shot-classification",model="facebook/bart-large-mnli")
classification(
    "This is a course about the Transformers library",
    candidate_labels=["education", "technology", "lifestyle"])

Device set to use cpu


{'sequence': 'This is a course about the Transformers library',
 'labels': ['technology', 'education', 'lifestyle'],
 'scores': [0.5308338403701782, 0.40530988574028015, 0.06385625898838043]}

In [ ]:
! pip install langchain_community

In [ ]:
from langchain_community.document_loaders import YoutubeLoader

In [ ]:
%pip install --upgrade --quiet  youtube-transcript-api

In [ ]:
loader = YoutubeLoader.from_youtube_url(
    "https://www.youtube.com/watch?v=vA0C0k72-b4", add_video_info=False
)

In [ ]:
docs= loader.load()

In [ ]:
#let's chat with this data

!pip install langchain langchain_community langchain_chroma

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)


In [ ]:
!pip install -q transformers sentence-transformers


In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings

# Load free Hugging Face embedding model
embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

/tmp/ipython-input-16-2207856108.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


In [ ]:
from langchain_chroma import Chroma

# Store the documents in Chroma vectorstore
vectorstore = Chroma.from_documents(documents=docs,embedding=embedding)
retriever = vectorstore.as_retriever()


In [ ]:
from transformers import pipeline
from langchain_community.llms import HuggingFacePipeline

# Load a free Hugging Face model for text generation
hf_pipe = pipeline("text2text-generation", model="google/flan-t5-large", max_length=512)
llm = HuggingFacePipeline(pipeline=hf_pipe)

Device set to use cpu


In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain.chains import RetrievalQA

# Prompt format for the chain
prompt = PromptTemplate.from_template("""
Use the context below to answer the user's question.

Context:
{context}

Question:
{question}

Answer:
""")

# Create QA Chain using retriever and Hugging Face LLM
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt}
)


In [ ]:
# Ask any question based on transcript
response = qa_chain.invoke({"query": "What is the video about?"})
print("Answer:", response["result"])


Token indices sequence length is longer than the specified maximum sequence length for this model (2967 > 512). Running this sequence through the model will result in indexing errors


Answer: How to deploy a machine learning model into a container using Docker and fast API


In [ ]:
qa_chain.invoke({"query": "What tools or techniques are mentioned in the video?"})


Token indices sequence length is longer than the specified maximum sequence length for this model (2971 > 512). Running this sequence through the model will result in indexing errors


{'query': 'What tools or techniques are mentioned in the video?',
 'result': 'Docker and fast API'}

In [ ]:
qa_chain.invoke({"query": "provide summary?"})


{'query': 'provide summary?',
 'result': "We'll learn how to deploy a machine learning model into a container, create an inference API, and test it through an interface and programmatically."}

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

# Define your system prompt for technical Q&A
system_prompt = (
    "You are a data science expert answering technical questions. "
    "Use the following retrieved context to answer the question. "
    "If you don't know the answer, say so. Answer only using the provided context.\n\n"
    "{context}"
)

# Create chat-style prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}")
])

# Create a chain that combines docs using the LLM and the prompt
question_answer_chain = create_stuff_documents_chain(llm, prompt)

# Create final retrieval + answering chain (RAG)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)


In [ ]:
response = rag_chain.invoke({"input": "Share the key data sources mentioned in the video."})
print("Answer:", response["answer"])


Token indices sequence length is longer than the specified maximum sequence length for this model (2996 > 512). Running this sequence through the model will result in indexing errors


Answer: Psychic learn
